In [2]:
#Initialize the kaggle and the dataset
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! touch kaggle.json | echo '{"username":"jundifathan","key":"d8ce5ff328ebb2f5dab22dc29dbd4546"}' > kaggle.json
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d rizwan123456789/potato-disease-leaf-datasetpld

In [ ]:
! unzip potato-disease-leaf-datasetpld.zip

In [ ]:
!pip install --upgrade keras

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
%matplotlib inline
import tensorflow as tf
import keras
import glob
import cv2
import pickle, datetime

from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D,GlobalAveragePooling2D
from keras.layers import LSTM, Input, TimeDistributed,Convolution2D,Activation
from keras.layers.convolutional import ZeroPadding2D
from keras.optimizers import RMSprop, SGD
from tensorflow.keras.layers import BatchNormalization
from keras.utils import np_utils
from keras import optimizers
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.preprocessing.image import array_to_img
from keras.models import load_model

from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import os

In [ ]:
dir_val = "/content/PLD_3_Classes_256/Validation/*"
dir_tra = "/content/PLD_3_Classes_256/Training/*"

In [3]:
#Create Training Model
train_leaves_images = []
train_leaves_labels = []
for directory_path in glob.glob(dir_tra):
    leaves_label = directory_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (32, 32))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_leaves_images.append(img)
        train_leaves_labels.append(leaves_label)
train_leaves_images = np.array(train_leaves_images)
train_leaves_labels = np.array(train_leaves_labels)

train_leaves_labels

NameError: ignored

In [ ]:
label_to_id = {v:i for i,v in enumerate(np.unique(train_leaves_labels))}
id_to_label = {v: k for k, v in label_to_id.items()}
train_label_ids = np.array([label_to_id[x] for x in train_leaves_labels])

In [ ]:
train_leaves_images.shape, train_label_ids.shape, train_leaves_labels.shape

In [ ]:
# Create Testing Model
test_leaves_images = []
test_leaves_labels = []
for directory_path in glob.glob(dir_val):
    leaves_label = directory_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)

        img = cv2.resize(img, (32, 32))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        test_leaves_images.append(img)
        test_leaves_labels.append(leaves_label)
test_leaves_images = np.array(test_leaves_images)
test_leaves_labels = np.array(test_leaves_labels)

In [ ]:
label_to_ids = {v:i for i,v in enumerate(np.unique(test_leaves_labels))}
id_to_labels = {v: k for k, v in label_to_id.items()}
test_label_ids = np.array([label_to_ids[x] for x in test_leaves_labels])

In [ ]:
test_leaves_images.shape, test_label_ids.shape

In [ ]:
x_train, y_train, x_test, y_test, N_CATEGORY =train_leaves_images,train_leaves_labels,test_leaves_images,test_leaves_labels,len(label_to_id)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape, N_CATEGORY)

In [ ]:
id_to_label

In [ ]:
#Create the CNN Architecture
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def get_CNN(input_shape, num_classes):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    return model


In [ ]:
CNN = get_CNN((32,32,3),N_CATEGORY)
CNN.summary()

In [ ]:
#Normalization of the images and one-hot encoding of the labels
CNN.compile(loss='categorical_crossentropy', optimizer=RMSprop(),metrics=['accuracy'])
X_normalized = np.array(x_train / 255.0 - 0.5 )
X_normalized_test = np.array(x_test / 255.0 - 0.5 )

label_binarizer = LabelBinarizer()
y_one_hot = label_binarizer.fit_transform(y_train)
y_one_hot_test = label_binarizer.fit_transform(y_test)

In [ ]:
import matplotlib.pyplot as plt

# Training the model and storing the history
Model_Train = CNN.fit(X_normalized, y_one_hot, batch_size=32, epochs=10, verbose=1, validation_data=(X_normalized_test, y_one_hot_test))

# Retrieving the training and validation loss values
train_loss = Model_Train.history['loss']
val_loss = Model_Train.history['val_loss']

# Retrieving the training and validation accuracy values
train_accuracy = Model_Train.history['accuracy']
val_accuracy = Model_Train.history['val_accuracy']

# Plotting the training and validation loss
plt.plot(range(1, len(train_loss) + 1), train_loss, 'bo-', label='Training Loss')
plt.plot(range(1, len(val_loss) + 1), val_loss, 'ro-', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.figure()

# Plotting the training and validation accuracy
plt.plot(range(1, len(train_accuracy) + 1), train_accuracy, 'bo-', label='Training Accuracy')
plt.plot(range(1, len(val_accuracy) + 1), val_accuracy, 'ro-', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


In [ ]:
#Save the CNN model for the future use(as it takes hours to be trained!)
CNN.save('CNNleaf.h5')

In [ ]:
#Pick the first Fully-Connected layer as the features which will be of dimension (1 x 256)
layer_name = 'dense_2'
FC_layer_model = Model(inputs=CNN.input,
                                 outputs=CNN.get_layer(layer_name).output)

In [ ]:
#Find the Features for n number of train images and we will get n x 256
#This means we will get 256 features for each images.
i=0
features=np.zeros(shape=(x_train.shape[0],256))
for directory_path in glob.glob(dir_tra):
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (32,32))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        img = np.expand_dims(img, axis=0)
        FC_output = FC_layer_model.predict(img)
        features[i]=FC_output
        i+=1

In [ ]:
#Save the features of the train images to use it in future.
np.save('features', features)

In [ ]:
#Name the feature rows as f_0, f_1, f_2...
feature_col=[]
for i in range(256):
    feature_col.append("f_"+str(i))
    i+=1


In [ ]:
#Create DataFrame with features and coloumn name
train_features=pd.DataFrame(data=features,columns=feature_col)
feature_col = np.array(feature_col)

train_class = list(np.unique(train_label_ids))
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_label_ids.shape)
train_class

In [ ]:
#Feed the extracted features with the labels to RANDOM FOREST
rf = RandomForestClassifier(n_estimators = 50, random_state = 40, max_features=5)

rf.fit(train_features, train_label_ids)

In [ ]:
#Find the Features from CNN's FC layer for n number of test images and we will get n x 256
i=0
features_test=np.zeros(shape=(y_test.shape[0],256))
for directory_path in glob.glob(dir_val):
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (32,32))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        img = np.expand_dims(img, axis=0)
        FC_output = FC_layer_model.predict(img)
        features_test[i]=FC_output
        i+=1

In [ ]:
#Create DataFrame with features and column name
test_features=pd.DataFrame(data=features_test,columns=feature_col)
feature_col = np.array(feature_col)

print('Test Features Shape:', test_features.shape)
print('Test Labels Shape:', test_label_ids.shape)

In [ ]:
#Feed the features of the test images to Random Forest Classifier to predict its class
predictions = rf.predict(test_features)

In [ ]:
accuracy=accuracy_score(predictions , test_label_ids)
print('Accuracy:', accuracy*100, '%.')

In [ ]:
from sklearn import tree


tree_estimator = rf.estimators_[0]


plt.figure(figsize=(10, 10))
tree.plot_tree(tree_estimator, filled=True)
plt.show()